<a href="https://colab.research.google.com/github/woongjoonchoi/Tutorials/blob/main/Pytorch/PytorchDatasetapi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import math

In [3]:
import time

In [4]:
import numpy as np
from matplotlib import pyplot as plt
from  itertools import count , islice
from matplotlib.patches import Rectangle

# IterableDataset Practice

## IterableDataset Duplicate

In [ ]:
class MyIterableDataset(torch.utils.data.IterableDataset) :
    def __init__(self,start, end) :
        super(MyIterableDataset).__init__()
        assert end>start , "this example code only works with end >=start"

        self.start = start
        self.end = end
    def __iter__(self) :
        worker_info = torch.utils.data.get_worker_info()
        worker_id = worker_info.id
        print(range(self.start,self.end))
        return iter(range(self.start,self.end))



In [ ]:
ds = MyIterableDataset(start=3 ,end =100)

In [ ]:
print(list(torch.utils.data.DataLoader(ds,num_workers=2)))

range(3, 100)
range(3, 100)
[tensor([3]), tensor([3]), tensor([4]), tensor([4]), tensor([5]), tensor([5]), tensor([6]), tensor([6]), tensor([7]), tensor([7]), tensor([8]), tensor([8]), tensor([9]), tensor([9]), tensor([10]), tensor([10]), tensor([11]), tensor([11]), tensor([12]), tensor([12]), tensor([13]), tensor([13]), tensor([14]), tensor([14]), tensor([15]), tensor([15]), tensor([16]), tensor([16]), tensor([17]), tensor([17]), tensor([18]), tensor([18]), tensor([19]), tensor([19]), tensor([20]), tensor([20]), tensor([21]), tensor([21]), tensor([22]), tensor([22]), tensor([23]), tensor([23]), tensor([24]), tensor([24]), tensor([25]), tensor([25]), tensor([26]), tensor([26]), tensor([27]), tensor([27]), tensor([28]), tensor([28]), tensor([29]), tensor([29]), tensor([30]), tensor([30]), tensor([31]), tensor([31]), tensor([32]), tensor([32]), tensor([33]), tensor([33]), tensor([34]), tensor([34]), tensor([35]), tensor([35]), tensor([36]), tensor([36]), tensor([37]), tensor([37]), tenso

## IterableDataset Remove Duplicate

### sol1 : Using get_worker_info()

In [ ]:
class MyIterableDataset(torch.utils.data.IterableDataset) :
    def __init__(self, start, end) :
        super(MyIterableDataset).__init__()
        assert end > start , "this example code only works with end >= start"

        self.start = start
        self.end = end
    
    def __iter__(self) :
        worker_info= torch.utils.data.get_worker_info()
        if worker_info is None :
            iter_start = self.start
            iter_end = self.end
        else :
            per_worker = int(math.ceil((self.end-self.start) ) /
                             float(worker_info.num_workers))
            worker_id = worker_info.id
            iter_start = self.start + worker_id * per_worker
            iter_end =  min(iter_start + per_worker , self.end)
            print(f'worker_id : {worker_id} \n iter_start : {iter_start}  iter_end : {iter_end}\n')
        return iter(range(iter_start, iter_end))


In [ ]:
ds = MyIterableDataset(start = 3 ,end = 100) 

In [ ]:
print(list(torch.utils.data.DataLoader(ds,num_workers=3)))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


worker_id : 0 
 iter_start : 3  iter_end : 35
worker_id : 1 
 iter_start : 35  iter_end : 67


worker_id : 2 
 iter_start : 67  iter_end : 99

[tensor([3]), tensor([35]), tensor([67]), tensor([4]), tensor([36]), tensor([68]), tensor([5]), tensor([37]), tensor([69]), tensor([6]), tensor([38]), tensor([70]), tensor([7]), tensor([39]), tensor([71]), tensor([8]), tensor([40]), tensor([72]), tensor([9]), tensor([41]), tensor([73]), tensor([10]), tensor([42]), tensor([74]), tensor([11]), tensor([43]), tensor([75]), tensor([12]), tensor([44]), tensor([76]), tensor([13]), tensor([45]), tensor([77]), tensor([14]), tensor([46]), tensor([78]), tensor([15]), tensor([47]), tensor([79]), tensor([16]), tensor([48]), tensor([80]), tensor([17]), tensor([49]), tensor([81]), tensor([18]), tensor([50]), tensor([82]), tensor([19]), tensor([51]), tensor([83]), tensor([20]), tensor([52]), tensor([84]), tensor([21]), tensor([53]), tensor([85]), tensor([22]), tensor([54]), tensor([86]), tensor([23]), tensor([5

### sol2 : Using worker_init_fn(worker_id)

In [ ]:
class MyIterableDataset(torch.utils.data.IterableDataset) :
    def __init__(self,start, end) :
        super(MyIterableDataset).__init__()
        assert end>start , "this example code only works with end >=start"

        self.start = start
        self.end = end
    def __iter__(self) :
        worker_info = torch.utils.data.get_worker_info()
        worker_id = worker_info.id
        print(f'worker_id : {worker_id} \n iter_start : {self.start}  iter_end : {self.end}\n')
        return iter(range(self.start,self.end))



In [ ]:
def worker_init_fn(worker_id) :
    worker_info = torch.utils.data.get_worker_info()
    dataset = worker_info.dataset
    overall_start = dataset.start
    overall_end = dataset.end

    per_worker = int(math.ceil((overall_end-overall_start) ) /
                     float(worker_info.num_workers))
    worker_id = worker_info.id
    dataset.start = overall_start + worker_id * per_worker
    dataset.end = min(dataset.start + per_worker , overall_end)
    # print(f'worker_id : {worker_id} , worker_start : {dataset.start}  ,  worekr_end : {dataset.end}')

In [ ]:
ds = MyIterableDataset(start = 3 ,end = 100) 

In [ ]:
print(list(torch.utils.data.DataLoader(ds,num_workers=2)))

worker_id : 0 
 iter_start : 3  iter_end : 100

worker_id : 1 
 iter_start : 3  iter_end : 100

[tensor([3]), tensor([3]), tensor([4]), tensor([4]), tensor([5]), tensor([5]), tensor([6]), tensor([6]), tensor([7]), tensor([7]), tensor([8]), tensor([8]), tensor([9]), tensor([9]), tensor([10]), tensor([10]), tensor([11]), tensor([11]), tensor([12]), tensor([12]), tensor([13]), tensor([13]), tensor([14]), tensor([14]), tensor([15]), tensor([15]), tensor([16]), tensor([16]), tensor([17]), tensor([17]), tensor([18]), tensor([18]), tensor([19]), tensor([19]), tensor([20]), tensor([20]), tensor([21]), tensor([21]), tensor([22]), tensor([22]), tensor([23]), tensor([23]), tensor([24]), tensor([24]), tensor([25]), tensor([25]), tensor([26]), tensor([26]), tensor([27]), tensor([27]), tensor([28]), tensor([28]), tensor([29]), tensor([29]), tensor([30]), tensor([30]), tensor([31]), tensor([31]), tensor([32]), tensor([32]), tensor([33]), tensor([33]), tensor([34]), tensor([34]), tensor([35]), tensor(

In [ ]:
print(list(torch.utils.data.DataLoader(ds,num_workers=10,worker_init_fn=worker_init_fn)))


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


worker_id : 0 
 iter_start : 3  iter_end : 12

worker_id : 1 
 iter_start : 12  iter_end : 21
worker_id : 2 
 iter_start : 21  iter_end : 30

worker_id : 3 
 iter_start : 30  iter_end : 39


worker_id : 4 
 iter_start : 39  iter_end : 48

worker_id : 5 
 iter_start : 48  iter_end : 57

worker_id : 6 
 iter_start : 57  iter_end : 66

worker_id : 7 
 iter_start : 66  iter_end : 75

worker_id : 8 
 iter_start : 75  iter_end : 84

worker_id : 9 
 iter_start : 84  iter_end : 93

[tensor([3]), tensor([12]), tensor([21]), tensor([30]), tensor([39]), tensor([48]), tensor([57]), tensor([66]), tensor([75]), tensor([84]), tensor([4]), tensor([13]), tensor([22]), tensor([31]), tensor([40]), tensor([49]), tensor([58]), tensor([67]), tensor([76]), tensor([85]), tensor([5]), tensor([14]), tensor([23]), tensor([32]), tensor([41]), tensor([50]), tensor([59]), tensor([68]), tensor([77]), tensor([86]), tensor([6]), tensor([15]), tensor([24]), tensor([33]), tensor([42]), tensor([51]), tensor([60]), tensor

# Sampler 

## Integer indicies sampler

In [ ]:
a = np.arange(100)

In [ ]:
class MySamplerTestDataset(torch.utils.data.Dataset) :

    def __init__(self , inp):
        self.inp  =inp

    def __getitem__(self , idx) :
        return self.inp[idx]
        ...
    def __len__(self):

        return len(self.inp)

    

In [ ]:
IntegerIndiciesSamplerdataset = MySamplerTestDataset(a)

In [ ]:
Randomsampler = torch.utils.data.RandomSampler(IntegerIndiciesSamplerdataset)

In [ ]:
print(list(torch.utils.data.DataLoader(IntegerIndiciesSamplerdataset , sampler = Randomsampler)))

[tensor([73]), tensor([42]), tensor([5]), tensor([63]), tensor([79]), tensor([4]), tensor([22]), tensor([12]), tensor([30]), tensor([1]), tensor([48]), tensor([38]), tensor([19]), tensor([45]), tensor([11]), tensor([32]), tensor([34]), tensor([28]), tensor([7]), tensor([44]), tensor([60]), tensor([83]), tensor([14]), tensor([51]), tensor([74]), tensor([69]), tensor([31]), tensor([25]), tensor([98]), tensor([67]), tensor([20]), tensor([50]), tensor([53]), tensor([0]), tensor([87]), tensor([59]), tensor([43]), tensor([33]), tensor([65]), tensor([64]), tensor([56]), tensor([41]), tensor([21]), tensor([84]), tensor([92]), tensor([70]), tensor([81]), tensor([94]), tensor([77]), tensor([88]), tensor([29]), tensor([52]), tensor([13]), tensor([99]), tensor([91]), tensor([6]), tensor([55]), tensor([17]), tensor([96]), tensor([8]), tensor([57]), tensor([15]), tensor([76]), tensor([71]), tensor([97]), tensor([9]), tensor([78]), tensor([85]), tensor([89]), tensor([95]), tensor([82]), tensor([58]),

## NonIntegral indicies sampler

In [ ]:
a = np.arange(100)

In [ ]:
b = np.arange(100 , 200)

In [ ]:
c = np.arange(200,300)

In [ ]:
dic = {'a' : a, 'b' :b , 'c' : c}

In [ ]:
class MySamplerTestDataset(torch.utils.data.Dataset) :

    def __init__(self , inp):
        self.inp  =inp

    def __getitem__(self , idx) :
        print(f"idx : {idx}")
        return self.inp[idx]
        ...
    def __len__(self):

        return len(self.inp)

    

In [ ]:
class CustomSampler(torch.utils.data.Sampler) :
    def __init__(self, data_source) :
        self.data_source = data_source
    def __iter__(self) :
        print('hi')

        return iter(self.data_source)
        # return iter(['a' , 'b' , 'c'])
    def __len__(self) :
        return len(self.data_source)

In [ ]:
NonintegerSamplerTestDataset = MySamplerTestDataset(dic)

In [ ]:
nonintegersampler = CustomSampler(dic)

In [ ]:
print(list(torch.utils.data.DataLoader(NonintegerSamplerTestDataset , sampler = nonintegersampler)))

hi
idx : a
idx : b
idx : c
[tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
         36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
         54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71,
         72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89,
         90, 91, 92, 93, 94, 95, 96, 97, 98, 99]]), tensor([[100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113,
         114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127,
         128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141,
         142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
         156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
         170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183,
         184, 185, 186, 187, 188, 189, 19

## Iterable can not use sampler or batch sampler

In [ ]:
class MySamplerTestIterableDatset(torch.utils.data.IterableDataset) :
    def __init__(self,inp) :
        self.inp =inp 
    def __iter__(self) :
        return iter(self.inp)
    def __len__(self) :
        return len(self.inp)

In [ ]:
itercustomsamplerdataset = MySamplerTestIterableDatset(a)

### torch predefined sampler test

In [ ]:
itersampler  = torch.utils.data.RandomSampler(itercustomsampler)

In [ ]:
# iterable dataset에  sampler, batch_sampler ,batch_size를 적용하면 안된다.

print(list(torch.utils.data.DataLoader(itercustomsamplerdataset ,batch_size = 4, batch_sampler = itersampler)))

ValueError: ignored

In [ ]:
print(list(torch.utils.data.DataLoader(itercustomsamplerdataset ,batch_size = 4, sampler = itersampler)))

ValueError: ignored

### torch custom sampler test

In [ ]:
print(list(torch.utils.data.DataLoader(itercustomsamplerdataset , sampler = nonintegersampler)))

ValueError: ignored

In [ ]:
print(list(torch.utils.data.DataLoader(itercustomsamplerdataset , batch_sampler = nonintegersampler)))

ValueError: ignored

# Batch

## Automatic Batching

### Map-style dataset

In [ ]:
class MySamplerTestDataset(torch.utils.data.Dataset) :

    def __init__(self , inp):
        self.inp  =inp

    def __getitem__(self , idx) :
        return self.inp[idx]
        ...
    def __len__(self):

        return len(self.inp)

    

In [ ]:
a = np.arange(100)

In [ ]:
BatchtestDataset = MySamplerTestDataset(a)

In [ ]:
print(list(torch.utils.data.DataLoader(BatchtestDataset, batch_size = 6)))

[tensor([0, 1, 2, 3, 4, 5]), tensor([ 6,  7,  8,  9, 10, 11]), tensor([12, 13, 14, 15, 16, 17]), tensor([18, 19, 20, 21, 22, 23]), tensor([24, 25, 26, 27, 28, 29]), tensor([30, 31, 32, 33, 34, 35]), tensor([36, 37, 38, 39, 40, 41]), tensor([42, 43, 44, 45, 46, 47]), tensor([48, 49, 50, 51, 52, 53]), tensor([54, 55, 56, 57, 58, 59]), tensor([60, 61, 62, 63, 64, 65]), tensor([66, 67, 68, 69, 70, 71]), tensor([72, 73, 74, 75, 76, 77]), tensor([78, 79, 80, 81, 82, 83]), tensor([84, 85, 86, 87, 88, 89]), tensor([90, 91, 92, 93, 94, 95]), tensor([96, 97, 98, 99])]


In [ ]:
print(list(torch.utils.data.DataLoader(BatchtestDataset, batch_size = 6 ,drop_last = True)))

[tensor([0, 1, 2, 3, 4, 5]), tensor([ 6,  7,  8,  9, 10, 11]), tensor([12, 13, 14, 15, 16, 17]), tensor([18, 19, 20, 21, 22, 23]), tensor([24, 25, 26, 27, 28, 29]), tensor([30, 31, 32, 33, 34, 35]), tensor([36, 37, 38, 39, 40, 41]), tensor([42, 43, 44, 45, 46, 47]), tensor([48, 49, 50, 51, 52, 53]), tensor([54, 55, 56, 57, 58, 59]), tensor([60, 61, 62, 63, 64, 65]), tensor([66, 67, 68, 69, 70, 71]), tensor([72, 73, 74, 75, 76, 77]), tensor([78, 79, 80, 81, 82, 83]), tensor([84, 85, 86, 87, 88, 89]), tensor([90, 91, 92, 93, 94, 95])]


### Iterable Style dataset

In [ ]:
class MySamplerTestIterableDatset(torch.utils.data.IterableDataset) :
    def __init__(self,inp) :
        self.inp =inp 
    def __iter__(self) :
        return iter(self.inp)
    def __len__(self) :
        return len(self.inp)

In [ ]:
a = np.arange(100)

In [ ]:
BatchiterabletestDataset = MySamplerTestIterableDatset(a)

In [ ]:
print(list(torch.utils.data.DataLoader(BatchiterabletestDataset, batch_size = 10)))

[tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), tensor([10, 11, 12, 13, 14, 15, 16, 17, 18, 19]), tensor([20, 21, 22, 23, 24, 25, 26, 27, 28, 29]), tensor([30, 31, 32, 33, 34, 35, 36, 37, 38, 39]), tensor([40, 41, 42, 43, 44, 45, 46, 47, 48, 49]), tensor([50, 51, 52, 53, 54, 55, 56, 57, 58, 59]), tensor([60, 61, 62, 63, 64, 65, 66, 67, 68, 69]), tensor([70, 71, 72, 73, 74, 75, 76, 77, 78, 79]), tensor([80, 81, 82, 83, 84, 85, 86, 87, 88, 89]), tensor([90, 91, 92, 93, 94, 95, 96, 97, 98, 99])]


In [ ]:
a_s = np.random.shuffle(a)

In [ ]:
# print(np.split(a,10))

[array([86, 53, 58, 44, 25, 19, 45, 40,  8, 39]), array([90, 95, 94,  9,  5, 22, 70, 43, 14, 83]), array([92, 55, 96, 30,  7, 89, 42, 93, 74,  6]), array([17, 65, 57, 79, 69, 24, 29, 61, 11, 71]), array([13, 98, 34, 85, 16,  2,  3, 72, 50, 26]), array([76, 68, 27, 82, 66, 48, 12, 78, 77, 28]), array([23, 62, 73, 36, 99, 49, 84, 59,  0, 56]), array([75, 88, 60, 35, 81, 33,  1, 54, 87, 51]), array([38, 47, 67,  4, 20, 21, 31, 91, 18, 64]), array([63, 46, 41, 97, 10, 37, 80, 15, 52, 32])]


### Custom Batch Sampler

In [ ]:
class MySamplerTestDataset(torch.utils.data.Dataset) :

    def __init__(self , inp):
        self.inp  =inp

    def __getitem__(self , idx) :
        return self.inp[idx]
        ...
    def __len__(self):

        return len(self.inp)

    

In [ ]:
class CustomSampler(torch.utils.data.Sampler) :
    def __init__(self, data_source) :
        self.data_source = data_source
    def __iter__(self) :
        print('hi')

        return iter(np.split(self.data_source,10))
        # return iter(['a' , 'b' , 'c'])
    def __len__(self) :
        return len(self.data_source)

In [ ]:
a = np.arange(100)

In [ ]:
dds  = MySamplerTestDataset(a)

In [ ]:
customsampler = CustomSampler(a)

#### using sampler argument

In [ ]:
# 10개의 element를 가진 sequence를 2개의 batch 로 묶어서 tensor로 만들어서 return한다.

ddd = list(torch.utils.data.DataLoader(dds,sampler = customsampler , batch_size = 2 ))
for dd in ddd :
    print(dd)

hi
tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14, 15, 16, 17, 18, 19]])
tensor([[20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
        [30, 31, 32, 33, 34, 35, 36, 37, 38, 39]])
tensor([[40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
        [50, 51, 52, 53, 54, 55, 56, 57, 58, 59]])
tensor([[60, 61, 62, 63, 64, 65, 66, 67, 68, 69],
        [70, 71, 72, 73, 74, 75, 76, 77, 78, 79]])
tensor([[80, 81, 82, 83, 84, 85, 86, 87, 88, 89],
        [90, 91, 92, 93, 94, 95, 96, 97, 98, 99]])


#### using batch_sampler argument

In [ ]:
ddd = list(torch.utils.data.DataLoader(dds,batch_sampler = customsampler  ))

#for indices in batch_sampler:
    # yield collate_fn([dataset[i] for i in indices])
for dd in ddd :
    print(dd)

hi
tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
tensor([10, 11, 12, 13, 14, 15, 16, 17, 18, 19])
tensor([20, 21, 22, 23, 24, 25, 26, 27, 28, 29])
tensor([30, 31, 32, 33, 34, 35, 36, 37, 38, 39])
tensor([40, 41, 42, 43, 44, 45, 46, 47, 48, 49])
tensor([50, 51, 52, 53, 54, 55, 56, 57, 58, 59])
tensor([60, 61, 62, 63, 64, 65, 66, 67, 68, 69])
tensor([70, 71, 72, 73, 74, 75, 76, 77, 78, 79])
tensor([80, 81, 82, 83, 84, 85, 86, 87, 88, 89])
tensor([90, 91, 92, 93, 94, 95, 96, 97, 98, 99])


## Non Automatic Batching

### Custom Sampler

In [ ]:
dds  = MySamplerTestDataset(a)

In [ ]:
customsampler= CustomSampler(dic)

In [ ]:
# Pytorch
#for index in sampler:
    # yield collate_fn(dataset[index])
# index가 sequence type이 되어도 상관이 없다.
# collate_fn이 automatic_batch가 disabled 되면 default로 numpy array를 tensor로 변환하기에 
# numpy array에 대한 operation을 적용할 수 잇다. 

print(list(torch.utils.data.DataLoader(dds,sampler = customsampler)))

hi
[tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]), tensor([[10, 11, 12, 13, 14, 15, 16, 17, 18, 19]]), tensor([[20, 21, 22, 23, 24, 25, 26, 27, 28, 29]]), tensor([[30, 31, 32, 33, 34, 35, 36, 37, 38, 39]]), tensor([[40, 41, 42, 43, 44, 45, 46, 47, 48, 49]]), tensor([[50, 51, 52, 53, 54, 55, 56, 57, 58, 59]]), tensor([[60, 61, 62, 63, 64, 65, 66, 67, 68, 69]]), tensor([[70, 71, 72, 73, 74, 75, 76, 77, 78, 79]]), tensor([[80, 81, 82, 83, 84, 85, 86, 87, 88, 89]]), tensor([[90, 91, 92, 93, 94, 95, 96, 97, 98, 99]])]


### custom collate_fn

In [5]:
class MySamplerTestDataset(torch.utils.data.Dataset) :

    def __init__(self , inp ,max_len):
        self.inp  =inp
        self.max_len = max_len
    def __getitem__(self , idx) :
        return self.inp[idx]
        ...
    def __len__(self):

        return len(self.inp)

    

In [6]:
class CustomSampler(torch.utils.data.Sampler) :
    def __init__(self, data_source) :
        self.data_source = data_source
    def __iter__(self) :
        print('hi')

        return iter(self.data_source)
        # return iter(['a' , 'b' , 'c'])
    def __len__(self) :
        return len(self.data_source)

In [52]:
def custom_collate(dataset) :

    # batch size를 지정하지 않을 경우 batch size =  1 ,  pytorch  dataset의 iterable이 argument로 넘겨진다.
    # print(type(dataset))
    dataset[0] = np.pad(dataset[0],pad_width = (0,max_len-len(dataset[0])) , mode='constant' ,constant_values=-1)
    

    return dataset

In [50]:
def non_autobatch_custom_collate(dataset) :

    # batch size를 지정하지 않을 경우 batch size =  1 ,  pytorch  dataset의 iterable이 argument로 넘겨진다.
    # print(type(dataset))
    dataset = np.pad(dataset,pad_width = (0,max_len-len(dataset)) , mode='constant' ,constant_values=-1)
    

    return dataset

In [28]:
indicies = [ ]
d = 0
for x in range(14) :
    d+=x
    indicies.append(d)

In [29]:
a=np.arange(100)

In [30]:
new_a=  np.split(a,indicies)

In [21]:
print(type(new_a[3]))

<class 'numpy.ndarray'>


In [31]:
max_len=0
for k in new_a :
    max_len = max(len(k),max_len)

In [59]:
# 다른 모듈간에 code가  분산되어 있으면 의미 있는 코드이지만
# jupyter lab 환경상 module scope에 max_len이 존재하므로 reference가 가능함

custom_collate.max_len = max_len

In [54]:
dds = MySamplerTestDataset(new_a,max_len)

In [56]:
print("Automatic batching with custom collate fn")

for _dd in torch.utils.data.DataLoader(dds ,collate_fn= custom_collate) : 
    print(len(_dd[0])) 
    print(_dd)

Automatic batching with custom collate fn
13
[array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1])]
13
[array([ 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1])]
13
[array([ 1,  2, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1])]
13
[array([ 3,  4,  5, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1])]
13
[array([ 6,  7,  8,  9, -1, -1, -1, -1, -1, -1, -1, -1, -1])]
13
[array([10, 11, 12, 13, 14, -1, -1, -1, -1, -1, -1, -1, -1])]
13
[array([15, 16, 17, 18, 19, 20, -1, -1, -1, -1, -1, -1, -1])]
13
[array([21, 22, 23, 24, 25, 26, 27, -1, -1, -1, -1, -1, -1])]
13
[array([28, 29, 30, 31, 32, 33, 34, 35, -1, -1, -1, -1, -1])]
13
[array([36, 37, 38, 39, 40, 41, 42, 43, 44, -1, -1, -1, -1])]
13
[array([45, 46, 47, 48, 49, 50, 51, 52, 53, 54, -1, -1, -1])]
13
[array([55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, -1, -1])]
13
[array([66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, -1])]
13
[array([78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90])]
13
[array([91, 92, 93, 94, 95, 96, 97, 98, 99, -

In [60]:
print("Non Automatic Batching using custom collate_fn")

for _dd in torch.utils.data.DataLoader(dds ,batch_size = None  ,collate_fn= non_autobatch_custom_collate) : 
    print(len(_dd)) 
    print(_dd)

Non Automatic Batching using custom collate_fn
13
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
13
[ 0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
13
[ 1  2 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
13
[ 3  4  5 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
13
[ 6  7  8  9 -1 -1 -1 -1 -1 -1 -1 -1 -1]
13
[10 11 12 13 14 -1 -1 -1 -1 -1 -1 -1 -1]
13
[15 16 17 18 19 20 -1 -1 -1 -1 -1 -1 -1]
13
[21 22 23 24 25 26 27 -1 -1 -1 -1 -1 -1]
13
[28 29 30 31 32 33 34 35 -1 -1 -1 -1 -1]
13
[36 37 38 39 40 41 42 43 44 -1 -1 -1 -1]
13
[45 46 47 48 49 50 51 52 53 54 -1 -1 -1]
13
[55 56 57 58 59 60 61 62 63 64 65 -1 -1]
13
[66 67 68 69 70 71 72 73 74 75 76 77 -1]
13
[78 79 80 81 82 83 84 85 86 87 88 89 90]
13
[91 92 93 94 95 96 97 98 99 -1 -1 -1 -1]
